In [2]:
import numpy as np
from numba import njit
import math
from geo3d import norm_L2, normalized_vector, normalized_quat, quat_as_matrix, UnitFrame, Vector, Point
from typing import Tuple

vec = [1,2,3,4]
vec_tup = tuple(vec)
vec_np = np.array(vec)
vec_geo = Vector(vec)
mat_np = np.eye(4)

## Normalization

In [13]:
def norm_np(vec):
    return np.linalg.norm(vec)

def norm_py(vec):
    return sum(v**2 for v in vec)

@njit
def norm_jit_np(vec):
    return np.sum(vec**2)

@njit
def norm_L2_new(vec):
    s = 0
    for v in vec:
        s += v**2
    return math.sqrt(s)

@njit
def normalized_vector_array(vec) -> np.ndarray:
    """ Return unit vector
    
    by dividing by Euclidean (L2) norm
    
    Args:
        vec array with elements x,y,z
    
    Returns: 
        array shape (3,) vector divided by L2 norm
    """
    res = np.empty(3)
    n = norm_L2(vec)
    res[0] = vec[0] / n
    res[1] = vec[1] / n
    res[2] = vec[2] / n
    return res

@njit
def normalized_vector_tuple(vec) -> Tuple[float, float, float]:
    """ Return unit vector
    
    by dividing by Euclidean (L2) norm
    
    Args:
        vec array with elements x,y,z
    
    Returns: 
        array shape (3,) vector divided by L2 norm
    """
    n = norm_L2(vec)
    return (vec[0] / n, vec[1] / n, vec[2] / n)


@njit
def normalized_quat_new(quat):
    n = norm_L2(vec)
    nq = (quat[0]/n, quat[1]/n, quat[2]/n, quat[3]/n)
    return nq

@njit
def cast_vec_to_array(vec):
    a = np.empty(3)
    a[0] = vec[0]
    a[1] = vec[1]
    a[2] = vec[2]
    return a

In [12]:
np.empty(3, dtype=float)

array([0., 0., 0.])

In [3]:
%%timeit
norm_np(vec)

6.84 µs ± 181 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)


In [4]:
%%timeit
norm_py(vec)

1.79 µs ± 20.9 ns per loop (mean ± std. dev. of 7 runs, 1000000 loops each)


In [5]:
%%timeit
norm_jit_np(np.array(vec))

The slowest run took 13.50 times longer than the fastest. This could mean that an intermediate result is being cached.
4.13 µs ± 6.1 µs per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [19]:
%%timeit
normalized_vector_new(vec_tup)

605 ns ± 39.5 ns per loop (mean ± std. dev. of 7 runs, 1000000 loops each)


In [21]:
%%timeit
np.array(normalized_vector(vec_tup))

1.13 µs ± 28.8 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)


In [28]:
%%timeit
vec_geo.normalize()

1.69 µs ± 22.8 ns per loop (mean ± std. dev. of 7 runs, 1000000 loops each)


In [29]:
%%timeit
vec_np/norm_np(vec)

8.51 µs ± 66.7 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)


### methods in geo3d

In [6]:
%%timeit
norm_L2(vec_tup)

258 ns ± 1.95 ns per loop (mean ± std. dev. of 7 runs, 1000000 loops each)


In [7]:
%%timeit
normalized_vector(vec_tup)

306 ns ± 4.85 ns per loop (mean ± std. dev. of 7 runs, 1000000 loops each)


In [9]:
%%timeit
quat_as_matrix(vec_tup)

586 ns ± 3.26 ns per loop (mean ± std. dev. of 7 runs, 1000000 loops each)


## vectors manipulation

In [4]:
@njit
def add_vectors(v1,v2):
    return (v1[0]+v2[0], v1[1]+v2[1], v1[2]+v2[2])

@njit
def mult_vector_scalar(v,s):
    return (v[0]*s, v[1]*s, v[2]*s)

@njit
def dot_vectors(v1,v2):
    return v1[0]*v2[0] + v1[1]*v2[1] + v1[2]*v2[2]

@njit
def mult_mat_vec(m,v):
    return (
        dot_vectors(m[0], v),
        dot_vectors(m[1], v),
        dot_vectors(m[2], v)
    )

In [28]:
%%timeit
add_vectors(vec_tup, vec_tup)

429 ns ± 11.2 ns per loop (mean ± std. dev. of 7 runs, 1000000 loops each)


In [29]:
%%timeit
vec_np+vec_np

466 ns ± 15.3 ns per loop (mean ± std. dev. of 7 runs, 1000000 loops each)


In [31]:
%%timeit
mat_np@vec_np

1.64 µs ± 13.5 ns per loop (mean ± std. dev. of 7 runs, 1000000 loops each)


In [32]:
%%timeit
mult_vector_scalar(vec_np, 3)

320 ns ± 4.02 ns per loop (mean ± std. dev. of 7 runs, 1000000 loops each)


In [36]:
%%timeit
vec_np * 3

735 ns ± 8.17 ns per loop (mean ± std. dev. of 7 runs, 1000000 loops each)


In [40]:
%%timeit
dot_vectors(vec_np, vec_np)

403 ns ± 5.92 ns per loop (mean ± std. dev. of 7 runs, 1000000 loops each)


In [42]:
%%timeit
mult_mat_vec(mat_np, vec_np)

429 ns ± 3.02 ns per loop (mean ± std. dev. of 7 runs, 1000000 loops each)


In [43]:
%%timeit
mat_np @ vec_np

1.61 µs ± 6.1 ns per loop (mean ± std. dev. of 7 runs, 1000000 loops each)


In [7]:
%%timeit
Point.from_array(
    mat_np @ vec_np + vec_np, copy=False
)

3.08 µs ± 30.4 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)


In [17]:
%%timeit
for i in range(1000):
    Point(add_vectors(mult_mat_vec(mat_np, vec_np), vec_np))

2.35 ms ± 33.1 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [20]:
@njit
def trafo(rot, trans, p):
    return cast_to_array(add_vectors(mult_mat_vec(rot, p), trans))

In [24]:
%%timeit
Point.from_array(trafo(mat_np, vec_np, vec_np), copy=False)

1.27 µs ± 13.5 ns per loop (mean ± std. dev. of 7 runs, 1000000 loops each)


In [18]:
%%timeit
for i in range(1000):
    Point.from_array(cast_to_array(add_vectors(mult_mat_vec(mat_np, vec_np), vec_np)), copy=False)

1.92 ms ± 32.3 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


## Casting to array

In [52]:
%%timeit
np.array(vec)

912 ns ± 6.45 ns per loop (mean ± std. dev. of 7 runs, 1000000 loops each)


In [52]:
%%timeit
np.array(vec_np)

412 ns ± 2.25 ns per loop (mean ± std. dev. of 7 runs, 1000000 loops each)


In [53]:
%%timeit
np.asarray(vec_np)

288 ns ± 3.29 ns per loop (mean ± std. dev. of 7 runs, 1000000 loops each)


In [11]:
%%timeit
Vector.from_array(vec_np, copy=False)

340 ns ± 5.16 ns per loop (mean ± std. dev. of 7 runs, 1000000 loops each)


In [12]:
%%timeit
Vector(vec_np)

660 ns ± 4.6 ns per loop (mean ± std. dev. of 7 runs, 1000000 loops each)


In [50]:
%%timeit
cast_to_array(vec_np)

611 ns ± 51.5 ns per loop (mean ± std. dev. of 7 runs, 1000000 loops each)
